## Preprocess

####  <b>nltk</b>
- tokenize
- Stemming
- Lemmatization
- StopWord

### Intro
- pandas
- regular expression
- numpy
- Vectorizer:Count/TF-IDF
- NB

## Sentiment Analysis

In [163]:
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import numpy as np
import pandas as pd
import re
import itertools
from sklearn.metrics import accuracy_score
from nltk.stem.wordnet import WordNetLemmatizer

lem = WordNetLemmatizer()

In [164]:
dataa = 'imdb_labelled.txt'



texts = []
labels = []

with open(dataa) as f:
        for line in f:
            splitted = line.split()
            labels.append(int(splitted[-1]))
            texts.append(' '.join(splitted[:-1]))
            data = pd.DataFrame(columns=['text', 'label'])
            data['text'] = texts
            data['label'] = labels

In [165]:
data.describe()

,label
count,1000.00000
mean,0.50000
std,0.50025
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [166]:
plot=data.groupby('label')['text'].nunique()

In [167]:
plot

label
0    499
1    498
Name: text, dtype: int64

In [168]:
data

,text,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
5,"The rest of the movie lacks art, charm, meanin...",0
6,Wasted two hours.,0
7,Saw the movie today and thought it was a good ...,1
8,A bit predictable.,0
9,Loved the casting of Jimmy Buffet as the scien...,1


In [169]:
def clean(text):
    text=re.sub(r'[0-9]+', r'',text)
    text = re.sub(r'[^\w]', ' ', text)
    #word level tokenization
    txt = nltk.tokenize.word_tokenize(text)
    #Stop word Removal
    stop_words = set(stopwords.words('english'))
    txt=[word for word in txt if word not in stop_words]
    #Lemmatizer
    for j in range(len(txt)):
        txt[j] = lem.lemmatize(txt[j], "v")
    return " ".join(txt)

## Remove numbers/stopwords/other symbols
## Tokenizing/Lemmatizing

In [170]:
clean('All work and no play makes jack dull boy. All work and no play makes jack a dull boy. @&*!*&*!')

'All work play make jack dull boy All work play make jack dull boy'

In [171]:
data['text'] = data['text'].map(lambda x: clean(x))

In [172]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data.text, data.label, test_size=0.25, random_state=0)

In [173]:
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [177]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=3, max_df=0.9)

train_vectors = vectorizer.fit_transform(x_train)
test_vectors = vectorizer.transform(x_test)

In [178]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB(alpha=0.1)
model.fit(train_vectors, y_train)

predictions = model.predict(vectorizer.transform(x_test))
accuracy_score(y_test, predictions)

0.776

## Let's do actual prediction

In [179]:
text = ['the movie was so bad']
features = vectorizer.transform(text)
predictions = model.predict(features)
print(predictions)

[0]
